In [1]:
import numpy as np
import torch
import torch.utils.data as data
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim

In [2]:
data = np.load("permuted_mnist.npz")
permuted_x_train = data["train_images"]
y_train = data["train_labels"]
permuted_x_test = data["test_images"]
y_test = data["test_labels"]

print(data.files)
print(permuted_x_test.shape)

permuted_x_train, permuted_x_val, y_train, y_val = train_test_split(
    permuted_x_train, y_train, test_size=0.1, random_state=42
)

permuted_x_train = torch.from_numpy(permuted_x_train).float()
y_train = torch.from_numpy(y_train).long()
permuted_x_val = torch.from_numpy(permuted_x_val).float()
y_val = torch.from_numpy(y_val).long()
permuted_x_test = torch.from_numpy(permuted_x_test).float()
y_test = torch.from_numpy(y_test).long()

['train_images', 'train_labels', 'test_images', 'test_labels']
(10000, 28, 28)


In [3]:
class PermutedSimpleMLP(nn.Module):
    def __init__(self, input_size, num_classes, hidden_layers, hidden_neurons, dropout_rate=0.5):
        super(PermutedSimpleMLP, self).__init__()
        self.input_size = input_size
        self.num_classes = num_classes
        self.hidden_layers = hidden_layers
        self.hidden_neurons = hidden_neurons
        
        layers = []
        layers.append(nn.Linear(input_size, hidden_neurons[0]))
        layers.append(nn.ReLU())
        for i in range(1, hidden_layers):
            layers.append(nn.Linear(hidden_neurons[i-1], hidden_neurons[i]))
            layers.append(nn.ReLU())
        self.hidden = nn.Sequential(*layers)
        
        self.dropout = nn.Dropout(dropout_rate)
        self.fc_out = nn.Linear(hidden_neurons[-1], num_classes)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.hidden(x)
        x = self.dropout(x)
        x = self.fc_out(x)
        return x



In [4]:
input_size = 28 * 28
num_classes = 10
hidden_layers = 3
hidden_neurons = [256,256,256]
dropout_rate = 0.5
num_permutations = 10 
model = PermutedSimpleMLP(input_size, num_classes, hidden_layers, hidden_neurons, dropout_rate)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

num_epochs = 10
batch_size = 64

train_dataset = TensorDataset(permuted_x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = TensorDataset(permuted_x_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_dataset = TensorDataset(permuted_x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [6]:
for epoch in range(num_epochs):
    model.train()
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Training Accuracy: {accuracy:.2f}%")
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Validation Accuracy: {accuracy:.2f}%")
    
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Test Accuracy: {accuracy:.2f}%")

print("Training completed.")


Epoch 1/10, Training Accuracy: 91.12%
Epoch 1/10, Validation Accuracy: 95.33%
Epoch 1/10, Test Accuracy: 95.19%
Epoch 2/10, Training Accuracy: 95.33%
Epoch 2/10, Validation Accuracy: 95.77%
Epoch 2/10, Test Accuracy: 95.45%
Epoch 3/10, Training Accuracy: 96.18%
Epoch 3/10, Validation Accuracy: 96.65%
Epoch 3/10, Test Accuracy: 96.59%
Epoch 4/10, Training Accuracy: 96.69%
Epoch 4/10, Validation Accuracy: 96.83%
Epoch 4/10, Test Accuracy: 96.84%
Epoch 5/10, Training Accuracy: 96.92%
Epoch 5/10, Validation Accuracy: 97.07%
Epoch 5/10, Test Accuracy: 96.94%
Epoch 6/10, Training Accuracy: 97.30%
Epoch 6/10, Validation Accuracy: 97.25%
Epoch 6/10, Test Accuracy: 97.26%
Epoch 7/10, Training Accuracy: 97.45%
Epoch 7/10, Validation Accuracy: 97.00%
Epoch 7/10, Test Accuracy: 96.61%
Epoch 8/10, Training Accuracy: 97.71%
Epoch 8/10, Validation Accuracy: 97.25%
Epoch 8/10, Test Accuracy: 97.12%
Epoch 9/10, Training Accuracy: 97.79%
Epoch 9/10, Validation Accuracy: 96.78%
Epoch 9/10, Test Accuracy:

In [7]:
hidden_layers = [2,3]
hidden_neurons = [128,256]

for j in hidden_layers:
    for k in hidden_neurons:
        kk = [k]*j
        input_size = 28 * 28
        num_classes = 10
        dropout_rate = 0.5
        model = PermutedSimpleMLP(input_size, num_classes, hidden_layers = j, hidden_neurons = kk, dropout_rate = dropout_rate)
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss()
        num_epochs = 4
        for epoch in range(num_epochs):
            model.train()
            correct = 0
            total = 0
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            accuracy = 100 * correct / total
            print(f"Epoch {epoch+1}/{num_epochs}, Hidden layers: {j}, Hidden Neurons: {kk}, Training Accuracy: {accuracy:.2f}%")

            model.eval()
            correct = 0
            total = 0
            with torch.no_grad():
                for inputs, labels in test_loader:
                    outputs = model(inputs)
                    _, predicted = torch.max(outputs, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

            accuracy = 100 * correct / total
            print(f"Epoch {epoch+1}/{num_epochs}, Hidden layers: {j}, Hidden Neurons: {kk}, Test Accuracy: {accuracy:.2f}%")

        



Epoch 1/4, Hidden layers: 2, Hidden Neurons: [128, 128], Training Accuracy: 85.75%
Epoch 1/4, Hidden layers: 2, Hidden Neurons: [128, 128], Test Accuracy: 94.44%
Epoch 2/4, Hidden layers: 2, Hidden Neurons: [128, 128], Training Accuracy: 93.12%
Epoch 2/4, Hidden layers: 2, Hidden Neurons: [128, 128], Test Accuracy: 95.63%
Epoch 3/4, Hidden layers: 2, Hidden Neurons: [128, 128], Training Accuracy: 94.33%
Epoch 3/4, Hidden layers: 2, Hidden Neurons: [128, 128], Test Accuracy: 95.11%
Epoch 4/4, Hidden layers: 2, Hidden Neurons: [128, 128], Training Accuracy: 95.01%
Epoch 4/4, Hidden layers: 2, Hidden Neurons: [128, 128], Test Accuracy: 96.18%
Epoch 1/4, Hidden layers: 2, Hidden Neurons: [256, 256], Training Accuracy: 88.79%
Epoch 1/4, Hidden layers: 2, Hidden Neurons: [256, 256], Test Accuracy: 95.28%
Epoch 2/4, Hidden layers: 2, Hidden Neurons: [256, 256], Training Accuracy: 94.17%
Epoch 2/4, Hidden layers: 2, Hidden Neurons: [256, 256], Test Accuracy: 95.43%
Epoch 3/4, Hidden layers: 2,